# Rio de Janeiro hazard validation


This notebook presents a workflow for validating the Sfincs flood hazard maps of the Acari River Basin using **HydroFlows**, based on the floodmarks survey conducted for the January 2024 event. The folder structure is the same as in the other notebooks (the user is reffered to the `rio_risk_climate_strategies` notebook). Precipitation is applied uniformly across the Acari using values from station Iraja (ID 11) for the historical event.

In [ ]:
# Import packages
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog
from hydroflows.methods import (
    catalog,
    hazard_validation,
    historical_events,
    script,
    sfincs,
)

In [ ]:
# Define case name, root directory and logger

pwd = Path().resolve() # current directory
name = "local-hazard-validation"  # case name
setup_root = Path(pwd, "setups", "local")
pwd_rel = "../../"  # relative path from the case directory to the current file

# Setup the logger
logger = setuplog(level="INFO")

## Create the workflow

In this block the workflow configuration is specified and a HydroFlows workflow is created. The workflow takes as input the following:
- the region polygon of the Acari river basin
- the data catalog files describing all the input datasets
- the HydroMT configuration files
- the model executables

In addition some more general settings are specified (like the historical event name and start/end time) that are used in the methods below.

In [ ]:
# Setup the config file and data libs

# Config
config = WorkflowConfig(
    # general settings
    region=Path(pwd, "data/region.geojson"),
    catalog_path_global=Path(pwd, "data/global-data/data_catalog.yml"),
    catalog_path_local=Path(pwd, "data/local-data/data_catalog.yml"),
    plot_fig=True,
    # sfincs settings
    hydromt_sfincs_config=Path(setup_root, "hydromt_config/sfincs_config_default.yml"),
    sfincs_exe=Path(pwd, "bin/sfincs_v2.1.1/sfincs.exe"),
    depth_min=0.05,
    subgrid_output=True,  # sfincs subgrid output should exist since it is used to downscale the floodmap
    # historical events settings
    historical_events_dates={
        "event_january_2024": {
            "startdate": "2024-01-12 23:00",
            "enddate": "2024-01-14 21:00",
        },
    },
    # validation settings
    floodmarks_geom=Path(pwd, "data/local-data/floodmap/laminas_levantadas.gpkg"),
    waterlevel_col="altura_cm",
    waterlevel_unit="cm",
    cmap="PiYG",
    bins=[-2.5, -2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5],
    bmap="CartoDB.PositronNoLabels",
)

w = Workflow(
    config=config,
    name=name,
    root=setup_root,
)

#### Merging the global and local data catalogs

Both local and global information is needed to build the SFINCS model. For this reason, with the following method we merge the two data sources

In [ ]:
# Merge global and local data catalogs
merged_catalog_global_local = catalog.MergeCatalogs(
    catalog_path1=w.get_ref("$config.catalog_path_global"),
    catalog_path2=w.get_ref("$config.catalog_path_local"),
    merged_catalog_path=Path(pwd, "data/merged_data_catalog_local_global.yml"),
)
w.create_rule(merged_catalog_global_local, rule_id="merge_global_local_catalogs")

#### Build SFINCS

The following method builds the SFINCS model for the Acari River Basin. It takes as input the region geometry, the HydroMT configuration file and the merged global-local data catalog and it generates SFINCS models saved in the models dir.

In [ ]:
# Build SFINCS model for the Acari river basin
# - settings from the hydromt_config per strategy
# - data from the merged global-local catalog

# Sfincs build
sfincs_build = sfincs.SfincsBuild(
    region=w.get_ref("$config.region"),
    sfincs_root="models/sfincs_default",
    config=w.get_ref("$config.hydromt_sfincs_config"),
    catalog_path=merged_catalog_global_local.output.merged_catalog_path,
    plot_fig=w.get_ref("$config.plot_fig"),
    subgrid_output=w.get_ref("$config.subgrid_output"),
)
w.create_rule(sfincs_build, rule_id="sfincs_build")

### Precipitation design events

Here, we preprocess the local precipitation file using the `preprocess_local_precip.py` script. This step generates a NetCDF file compatible with the HydroFlows `HistoricalEvents` method, which is used to derive the historical event, i.e. January's 2024, used for validation. The hisotrical event(s) is then saved in the event/historical directory.

In [ ]:
# Preprocess local precipitation data and get the historical event for validation, i.e January's 2024
# Preprocess precipitation
precipitation = script.ScriptMethod(
    script=Path(pwd, "scripts", "preprocess_local_precip.py"),
    output={
        "precip_nc": Path(
            pwd, "data/preprocessed-data/output_scalar_resampled_precip_station11.nc"
        )
    },
)
w.create_rule(precipitation, rule_id="preprocess_local_rainfall")

historical_event = historical_events.HistoricalEvents(
    precip_nc=precipitation.output.precip_nc,
    events_dates=w.get_ref("$config.historical_events_dates"),
    output_dir="events/historical",
    wildcard="pluvial_historical_event",
)
w.create_rule(historical_event, rule_id="historical_event")

### Derive flood hazard

In the following block, we derive the flood hazard for the historical event. The hazard derivation is performed using the `SfincsUpdateForcing`, `SfincsRun`, and `SfincsDownscale` methods. First, the SFINCS historical forcing is updated using `SfincsUpdateForcing`. Then, the model execution is carried out using the `SfincsRun` method and finally the `SfincsDownscale` method is used to generate high-resolution flood hazard maps from the SFINCS output.

In [ ]:
# Update the SFINCS model with the historical event
# This will create new SFINCS instances for the hisotrical event in the simulations subfolder
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_yaml=historical_event.output.event_yaml,
    output_dir=sfincs_build.output.sfincs_inp.parent
    / "simulations"
    / "{pluvial_historical_event}",
)
w.create_rule(sfincs_update, rule_id="sfincs_update")

# Run the SFINCS model for the historical event
# This will create simulated water levels for the historical event
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    sfincs_exe=w.get_ref("$config.sfincs_exe"),
)
w.create_rule(sfincs_run, rule_id="sfincs_run")

# Downscale the SFINCS output to derive high-res flood hazard maps
sfincs_down = sfincs.SfincsDownscale(
    sfincs_map=sfincs_run.output.sfincs_map,
    sfincs_subgrid_dep=sfincs_build.output.sfincs_subgrid_dep,
    depth_min=w.get_ref("$config.depth_min"),
    output_root="output/hazard_historical",
)
w.create_rule(sfincs_down, rule_id="sfincs_downscale")


# the simulation is stored in:
print("simulation folder:", sfincs_update.output.sfincs_out_inp.parent)

# the hazard map is stored in:
print("high-res hazard map folder:", sfincs_down.output.hazard_tif.parent)

### Hazard validation

In this step we validate the derived downscaled inundation map for the historical event against floodmarks. This is done by using the `FloodmarksValidation`.

In [ ]:
# Validate the downscaled inundation map against floodmarks
floodmark_validation = hazard_validation.FloodmarksValidation(
    floodmarks_geom=w.get_ref("$config.floodmarks_geom"),
    flood_hazard_map=sfincs_down.output.hazard_tif,
    waterlevel_col=w.get_ref("$config.waterlevel_col"),
    waterlevel_unit=w.get_ref("$config.waterlevel_unit"),
    out_root="output/validation/{pluvial_historical_event}",
    bmap=w.get_ref("$config.bmap"),
    bins=w.get_ref("$config.bins"),
    cmap=w.get_ref("$config.cmap"),
    region=sfincs_build.output.sfincs_region,
)
w.create_rule(floodmark_validation, rule_id="floodmark_validation")

## Visualize and execute the workflow

The workflow can be executed using HydroFlows or a workflow engine. Below we first plot and dryrun the workflow to check if it is correctly defined. Then, we parse the workflow to SnakeMake to execute it.

In [ ]:
w.plot_rulegraph()

In [ ]:
w.dryrun()

In [ ]:
# to snakemake
w.to_snakemake(f"{name}.smk")